In [3]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
# %matplotlib inline
%matplotlib qt5
import mne
mne.viz.set_browser_backend("qt")  # or "matplotlib"
mne.set_config("MNE_BROWSER_BACKEND", "qt")  # or "matplotlib"
%gui qt

import IPython

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using qt as 2D backend.



# Use MNE to load and analyze saved EEG and Motion recordings


In [2]:
import time
import re
from datetime import datetime, timezone

import uuid
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from nptyping import NDArray
from matplotlib import pyplot as plt

from pathlib import Path
import numpy as np
import pandas as pd
from numpy.typing import NDArray

import mne
from mne import set_log_level
from copy import deepcopy
import mne

from mne.io import read_raw

datasets = []
# mne.viz.set_browser_backend("Matplotlib")
mne.viz.set_browser_backend("qt")

from mne_lsl.player import PlayerLSL as Player
from mne_lsl.stream import StreamLSL as Stream

from phoofflineeeganalysis.EegProcessing import bandpower
from phoofflineeeganalysis.analysis.MNE_helpers import MNEHelpers
from phoofflineeeganalysis.analysis.historical_data import HistoricalData
from phoofflineeeganalysis.analysis.motion_data import MotionData
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData
from phoofflineeeganalysis.analysis.anatomy_and_electrodes import ElectrodeHelper
# from ..EegProcessing import bandpower
from phoofflineeeganalysis.EegProcessing import analyze_eeg_trends
from phoofflineeeganalysis.EegVisualization import VisHelpers
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import SavedSessionsProcessor, SessionModality, DataModalityType, LabRecorderXDF, unwrap_single_element_listlike_if_needed
from phoofflineeeganalysis.analysis.historical_data import GenericRawDebugFileProcessor
from phoofflineeeganalysis.analysis.MNE_helpers import DatasetDatetimeBoundsRenderingMixin, RawArrayExtended, RawExtended, up_convert_raw_objects, up_convert_raw_obj

set_log_level("WARNING")

# eeg_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/fif').resolve()
# headset_motion_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/MOTION_RECORDINGS/fif').resolve()

# assert eeg_recordings_file_path.exists()
# assert headset_motion_recordings_file_path.exists()

eeg_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/fif').resolve()
flutter_eeg_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEEG_FlutterRecordings').resolve()
flutter_motion_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEEG_FlutterRecordings/MOTION_RECORDINGS').resolve()
flutter_GENERIC_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEEG_FlutterRecordings/GENERIC_RECORDINGS').resolve()

headset_motion_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/MOTION_RECORDINGS/fif').resolve()
WhisperVideoTranscripts_LSL_Converted = Path(r"E:/Dropbox (Personal)/Databases/UnparsedData/WhisperVideoTranscripts_LSL_Converted").resolve()
pho_log_to_LSL_recordings_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/PhoLogToLabStreamingLayer_logs').resolve()
## These contain little LSL .fif files with names like: '20250808_062814_log.fif', 

eeg_analyzed_parent_export_path = Path("E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed").resolve()
pickled_data_path = Path(r"E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed/PICKLED_COLLECTION").resolve()
assert pickled_data_path.exists()

# n_most_recent_sessions_to_preprocess: int = None # None means all sessions
# n_most_recent_sessions_to_preprocess: int = 55
n_most_recent_sessions_to_preprocess: int = 5
# n_most_recent_sessions_to_preprocess = None


ModuleNotFoundError: No module named 'mne'

In [2]:
## Flutter App EEG and Motion Recordings
from phoofflineeeganalysis.analysis.historical_data import HistoricalData, FlutterExportedData, GenericRawDebugFileProcessor

flat_data_modality_dict, found_recording_file_modality_dict = FlutterExportedData.try_load_flutter_recordings(should_load_data=False, should_process=True,
                                                                                                                # flutter_GENERIC_recordings_file_path=None, ## override
                                                                                                                n_most_recent_sessions_to_preprocess=n_most_recent_sessions_to_preprocess)
# found_recording_file_modality_dict
# flat_data_modality_dict

file_type: EEG PROCESSING -- found 61 files, loading only 5: 
FILE[0]: eeg_data_2025-09-18T21-05-57.180582 | file_type: "EEG" PROCESSING:
WARN: reordering columns!
FILE[1]: eeg_data_2025-09-18T21-01-27.830069 | file_type: "EEG" PROCESSING:
WARN: reordering columns!
FILE[2]: eeg_data_2025-09-18T20-58-22.908260 | file_type: "EEG" PROCESSING:
WARN: reordering columns!
FILE[3]: eeg_data_2025-09-18T20-58-22.478351 | file_type: "EEG" PROCESSING:
WARN: reordering columns!
	Encountered error: len(raw.times) == 0!! True while trying to read CSV file E:\Dropbox (Personal)\Databases\UnparsedData\EmotivEEG_FlutterRecordings\eeg_data_2025-09-18T20-58-22.478351.csv. Skipping
FILE[4]: eeg_data_2025-09-18T20-56-20.484420 | file_type: "EEG" PROCESSING:
WARN: reordering columns!
GENERIC FILE PROCESSING -- found 22 files, loading only 5: 
FILE[0]: generic_data_2025-09-18T21-05-57.225804 | file_type: "GENERIC" PROCESSING:
GENERIC FILE PROCESSING
Performing one-time init of `cls.emotiv_epoc_x`...
crypto_ke

In [ ]:
flat_data_modality_dict

In [ ]:
found_recording_file_modality_dict

In [3]:
modality_datasets, modality_df = flat_data_modality_dict['GENERIC']
motion_datasets = modality_datasets['MOTION']
eeg_datasets = modality_datasets['EEG']
# motion_datasets = up_convert_raw_objects(modality_datasets['MOTION'])
# eeg_datasets = up_convert_raw_objects(modality_datasets['EEG'])


In [6]:
eeg_datasets

[<RawArrayExtended | 15 x 39907 (311.8 s), ~4.6 MB, data loaded>, timerange=2025-09-18T21:05:57+00:00 → 2025-09-18T21:11:08.765625+00:00,
 <RawArrayExtended | 15 x 530 (4.1 s), ~82 kB, data loaded>, timerange=2025-09-18T21:01:27+00:00 → 2025-09-18T21:01:31.132812+00:00,
 <RawArrayExtended | 15 x 11712 (91.5 s), ~1.4 MB, data loaded>, timerange=2025-09-18T20:58:22+00:00 → 2025-09-18T20:59:53.492188+00:00,
 <RawArrayExtended | 15 x 5669 (44.3 s), ~684 kB, data loaded>, timerange=2025-09-18T20:56:20+00:00 → 2025-09-18T20:57:04.281250+00:00]

In [ ]:
eeg_datasets[0]

In [4]:
## Original EEG
flutter_decoded_eeg_datasets, flutter_decoded_eeg_df = flat_data_modality_dict['EEG'] ## these were the eeg-specific files written by flutter to the non-generic .csv after decoding. They should be compared to the values returned from the GENERIC eeg
# list(flutter_decoded_eeg_datasets.values())[0]



In [ ]:
flutter_decoded_eeg_datasets['EEG']


[<RawArrayExtended | 15 x 80123 (626.0 s), ~9.2 MB, data loaded>, timerange=2025-09-18T21:05:57+00:00 → 2025-09-18T21:16:22.953125+00:00,
 <RawArrayExtended | 15 x 1009 (7.9 s), ~138 kB, data loaded>, timerange=2025-09-18T21:01:27+00:00 → 2025-09-18T21:01:34.875000+00:00,
 <RawArrayExtended | 15 x 21282 (166.3 s), ~2.5 MB, data loaded>, timerange=2025-09-18T20:58:22+00:00 → 2025-09-18T21:01:08.257812+00:00,
 <RawArrayExtended | 15 x 10357 (80.9 s), ~1.2 MB, data loaded>, timerange=2025-09-18T20:56:20+00:00 → 2025-09-18T20:57:40.906250+00:00]

## Can finally do the comparison!!

In [24]:
manual_decrypted_raw = deepcopy(eeg_datasets[0]).copy().pick_types(eeg=True, meg=False, stim=False, exclude='bads')
flutter_decoded_raw = deepcopy(flutter_decoded_eeg_datasets['EEG'][0]).copy().pick_types(eeg=True, meg=False, stim=False, exclude='bads')

manual_decrypted_raw
flutter_decoded_raw

<RawArrayExtended | 14 x 39907 (311.8 s), ~4.3 MB, data loaded>, timerange=2025-09-18T21:05:57+00:00 → 2025-09-18T21:11:08.765625+00:00

<RawArrayExtended | 14 x 80123 (626.0 s), ~8.6 MB, data loaded>, timerange=2025-09-18T21:05:57+00:00 → 2025-09-18T21:16:22.953125+00:00

In [25]:
mne.viz.set_browser_backend("qt")  # or "matplotlib"
mne.set_config("MNE_BROWSER_BACKEND", "qt")  # or "matplotlib"


'qt'

In [26]:

plot_kwargs = dict(start=10.0, duration=30.0, scalings=dict(eeg='auto'), time_format='clock')
manual_decrypted_raw.plot(title='manual_decrypted', **plot_kwargs)

In [27]:
flutter_decoded_raw.plot(title='flutter_decoded', **plot_kwargs)

In [ ]:
# type(flutter_decoded_eeg_datasets)
print(list(flutter_decoded_eeg_datasets.keys()))

In [ ]:
a_raw = eeg_datasets[-1]
a_raw

In [ ]:
a_raw.plot()

In [ ]:

type(flutter_decoded_eeg_datasets)


In [ ]:
motion_datasets[-2].plot()
# eeg_datasets[-2].plot()

In [ ]:
from phoofflineeeganalysis.analysis.motion_data import MotionData
from phoofflineeeganalysis.analysis.MNE_helpers import MNEHelpers


## INPUTS: eeg_datasets, motion_datasets
## set the annotations for the EEG-type modalities

for an_eeg_ds in eeg_datasets:
    # ==================================================================================================================================================================================================================================================================================== #
    # Adding `DataModalityType.PHO_LOG_TO_LSL` before `DataModalityType.MOTION` annotations works, while the opposite order seems to lose the MOTION annotations                                                                                                                           #
    # ==================================================================================================================================================================================================================================================================================== #
    # for an_annotation_ds in raws_dict.get(DataModalityType.PHO_LOG_TO_LSL.value, []):
    #     # meas_date = deepcopy(an_eeg_ds.info.get('meas_date'))
    #     # MNEHelpers.merge_annotations(raw=an_eeg_ds, new_annots=an_annotation_ds, align_to_Raw_meas_time=True)
    #     MNEHelpers.merge_annotations(raw=an_eeg_ds, new_annots=an_annotation_ds, align_to_Raw_meas_time=False)        
    #     # if (an_eeg_ds.annotations is None) or (len(an_eeg_ds.annotations) < 1):
    #     #     # an_eeg_ds.annotations = an_annotation_ds
    #     #     an_eeg_ds.set_annotations(an_annotation_ds)
    #     # else:
    #     #     # a_raw: mne.io.Raw = mne.io.Raw(an_eeg_ds)
    #     #     an_eeg_ds.set_annotations(an_annotation_ds)
    #     #     # an_eeg_ds.set_annotation(an_annotation_ds)
    #     # an_eeg_ds.annotations

    # ==================================================================================================================================================================================================================================================================================== #
    # Add Motion Annotations                                                                                                                                                                                                                                                               #
    # ==================================================================================================================================================================================================================================================================================== #
    for an_motion_raw_ds in motion_datasets:

        motion_annots: mne.Annotations = MotionData.find_high_accel_periods(an_motion_raw_ds, should_set_bad_period_annotations=True)

        motion_annots = MNEHelpers.merge_annotations(raw=an_eeg_ds, new_annots=motion_annots, align_to_Raw_meas_time=True)


## OUTPUTS: eeg_datasets
# eeg_datasets

In [ ]:
eeg_datasets

In [ ]:
from phoofflineeeganalysis.analysis.MNE_helpers import MNEHelpers
# [v.to_data_frame(time_format='datetime') for v in eeg_datasets]


# eeg_min_max_times_df = MNEHelpers.build_dataset_span_dataframe(a_raw_datasets=eeg_datasets)
motion_min_max_times_df = MNEHelpers.build_dataset_span_dataframe(a_raw_datasets=motion_datasets)
motion_min_max_times_df

In [ ]:
from datetime import timedelta

((motion_min_max_times_df['duration'] / np.timedelta64(1, 's')) < (timedelta(minutes=1) / np.timedelta64(1, 's')))



In [ ]:
# motion_min_max_times_df['duration'].dt.
motion_min_max_times_df['duration'].dt.total_seconds()

In [ ]:
motion_datasets

In [ ]:

raw_dfs_dict, min_max_times_dict = MNEHelpers.debug_compare_raw_alignments(time_col_name='time', eeg=eeg_datasets)
# raw_dfs_dict
min_max_times_dict



In [ ]:


a_motion_ds = motion_datasets[-1]
a_motion_df: pd.DataFrame = a_motion_ds.to_data_frame()
a_motion_df: pd.DataFrame = MotionData.compute_rolling_motion_change_detection(a_df=a_motion_df)

a_rolling_motion_df = a_motion_df.rolling(window=4, step=1).mean()
a_rolling_motion_df: pd.DataFrame = MotionData.compute_rolling_motion_change_detection(a_df=a_rolling_motion_df)


# a_motion_df: pd.DataFrame = MotionData.compute_rolling_motion_change_detection(a_df=a_motion_df)

# dataset_moving_annotations: mne.Annotations = MNEHelpers.convert_df_with_boolean_col_to_epochs(a_motion_df, is_bad_col_name="is_moving", annotation_description_name="BAD_motion", time_col_names='time')
dataset_moving_annotations: mne.Annotations = MNEHelpers.convert_df_with_boolean_col_to_epochs(a_rolling_motion_df, is_bad_col_name="is_moving", annotation_description_name="BAD_motion", time_col_names='time')
dataset_moving_annotations



In [ ]:
from phoofflineeeganalysis.analysis.MNE_helpers import MNEHelpers

a_ds = a_motion_ds

dataset_moving_annotations: mne.Annotations = MNEHelpers.convert_df_with_boolean_col_to_epochs(a_motion_df, is_bad_col_name="is_moving", annotation_description_name="BAD_motion", time_col_names='time')
dataset_moving_annotations

In [ ]:
a_ds.set_annotations(dataset_moving_annotations)

# a_ds.annotations

In [ ]:
is_bad = a_motion_df['is_moving']
a_ds = a_motion_ds

meas_date = a_ds.info['meas_date']
active_times = deepcopy(a_ds.times)


active_times

In [ ]:
a_motion_df['time'].to_numpy()


In [ ]:

## INPUTS: meas_date

active_times = deepcopy(a_motion_df['time'].to_numpy())

# split into contiguous segments
idx = np.where(np.diff(is_bad.astype(int)) != 0)[0] + 1 # the +1 is to adjust for the diff operation
segments = np.split(np.arange(len(is_bad)), idx)

onsets, durations = [], []
for seg in segments:
    if is_bad[seg[0]]:
        onsets.append(active_times[seg[0]])
        durations.append(active_times[seg[-1]] - active_times[seg[0]])

is_moving_annots = mne.Annotations(onset=onsets, duration=durations, description=["BAD_motion"]*len(onsets), orig_time=meas_date)
is_moving_annots



In [ ]:
a_rolling_motion_df: pd.DataFrame = MotionData.compute_rolling_motion_change_detection(a_df=a_rolling_motion_df)
a_rolling_motion_df

In [ ]:
motion_datasets[-1].plot()


In [ ]:
eeg_datasets[-1].plot()

In [ ]:
flat_data_modality_dict

In [ ]:

# flat_data_modality_dict['MOTION']
# (all_data_EEG, all_times_EEG), datasets_EEG, df_EEG = flat_data_modality_dict['EEG']  ## Unpacking

(all_data_EEG, all_times_EEG), datasets_EEG, df_EEG = flat_data_modality_dict['GENERIC']  ## Unpacking
# _, datasets, df = flat_data_modality_dict['MOTION'][-1]  ## Unpacking
# df_EEG

a_raw = datasets_EEG[-1]
# a_raw.plot(block=True)
a_raw.describe(data_frame=True)

In [ ]:

# (['eeg'] * (len(ch_names)-1) + ['misc'])

a_raw


In [ ]:
mne.viz.set_browser_backend("Matplotlib")

a_raw.pick_types(eeg=True).plot(block=True)


## 2025-09-11 - Almost seems working but see also
From `CyKit/Examples/example_epoc_plus.py`
```python
join_data = ''.join(map(chr, data[1:]))
data = self.cipher.decrypt(bytes(join_data,'latin-1')[0:32])
```

# SavedSessionProcessor

In [ ]:
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import SavedSessionsProcessor, SessionModality, DataModalityType

sso: SavedSessionsProcessor = SavedSessionsProcessor(eeg_recordings_file_path=eeg_recordings_file_path,
                                                     headset_motion_recordings_file_path=headset_motion_recordings_file_path, WhisperVideoTranscripts_LSL_Converted_file_path=WhisperVideoTranscripts_LSL_Converted, pho_log_to_LSL_recordings_path=pho_log_to_LSL_recordings_path,
                                                    eeg_analyzed_parent_export_path=eeg_analyzed_parent_export_path, 
                                                     n_most_recent_sessions_to_preprocess=n_most_recent_sessions_to_preprocess, 
                                                    should_load_data=True, should_load_preprocessed=False,
                                                    #  should_load_data=True, should_load_preprocessed=True,
													)

In [ ]:
(flat_data_modality_dict, found_recording_file_modality_dict) = sso.setup_specific_modality(modality_type=[DataModalityType.EEG, DataModalityType.MOTION, DataModalityType.PHO_LOG_TO_LSL, DataModalityType.WHISPER], should_load_data=False)

## 3m 0.3s

## 2025-09-09 - Manual Read to determine only the new/modified EEG files

In [ ]:
from phoofflineeeganalysis.analysis.historical_data import HistoricalData

updated_file_paths, (pending_updated_recording_file_df, modern_found_EEG_recording_file_df, pre_processed_EEG_recording_file_df) = HistoricalData.discover_updated_recording_files(eeg_recordings_file_path=sso.eeg_recordings_file_path, eeg_analyzed_parent_export_path=sso.eeg_analyzed_parent_export_path)
# updated_file_paths
flat_data_modality_dict = HistoricalData.read_recording_files(found_recording_file_modality_dict={DataModalityType.EEG.name: updated_file_paths}, should_load_data=True)
flat_data_modality_dict

In [ ]:
flat_data_modality_dict, found_recording_file_modality_dict = HistoricalData.MAIN_process_recording_files(
    eeg_recordings_file_path = sso.eeg_analyzed_parent_export_path,
    # headset_motion_recordings_file_path = self.headset_motion_recordings_file_path,
    # WhisperVideoTranscripts_LSL_Converted = self.WhisperVideoTranscripts_LSL_Converted_file_path,
    # pho_log_to_LSL_recordings_path = self.pho_log_to_LSL_recordings_path,
    should_load_data=False,
    # should_load_data=sso.should_load_data,
)



# 1m 7sec



# ## #TODO 2025-09-09 16:14: - [ ] Find the files that changed since last processing, and only load those:
# flat_data_modality_dict, found_recording_file_modality_dict = HistoricalData.MAIN_process_recording_files(
#     eeg_recordings_file_path = sso.eeg_recordings_file_path,
#     headset_motion_recordings_file_path = sso.headset_motion_recordings_file_path,
#     WhisperVideoTranscripts_LSL_Converted = sso.WhisperVideoTranscripts_LSL_Converted_file_path,
#     pho_log_to_LSL_recordings_path = sso.pho_log_to_LSL_recordings_path,
#     should_load_data=sso.should_load_data,
# )


In [ ]:

# prev_processed_modality_EEG: SessionModality = flat_data_modality_dict[DataModalityType.EEG.name]  ## Unpacking
_, prev_processed_datasets, prev_processed_df_EEG = flat_data_modality_dict[DataModalityType.EEG.name]  ## Unpacking
prev_processed_datasets
# prev_processed_df_EEG


[a_raw for a_raw in prev_processed_datasets]
a_raw = prev_processed_datasets[-1]
a_raw






In [ ]:
prev_processed_df_EEG: pd.DataFrame = prev_processed_modality_EEG.df  ## Unpacking
prev_processed_df_EEG['duration'] = prev_processed_df_EEG['end_time'] - prev_processed_df_EEG['start_time']
prev_processed_df_EEG



# Resume main pipleine

In [ ]:
sso.perform_post_processing() # Filename '2024-09-11 15-27-36.lsl.fif' does not contain a recognized datetime format.
## 3m 7.5s

In [ ]:
written_EDF_file_paths = sso.save_to_EDF()
written_EDF_file_paths

### Pickle Entire Pipeline

In [ ]:
# data_path = Path(r"C:/Users/pho/repos/EmotivEpoc/PhoLabStreamingReceiver/data").resolve()
# assert data_path.exists()

# pkl_path = pickled_data_path.joinpath("2025-09-02_50records_SSO_all.pkl").resolve()
# pkl_path = pickled_data_path.joinpath("2025-09-08_ALL_records_SSO_all.pkl").resolve()
pkl_path = pickled_data_path.joinpath("2025-09-10_35_records_SSO_all.pkl").resolve()
sso.save(pkl_path=pkl_path)

# 8m 20s

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates

## sso
(all_data_EEG, all_times_EEG), datasets_EEG, df_EEG = sso.flat_data_modality_dict[DataModalityType.EEG.name]  ## Unpacking
datasets_EEG

In [ ]:
df_EEG

In [ ]:
sso.modalities['EEG']

In [ ]:
## INPUTS: datasets_EEG

# Assume raws is your list of Raw objects
times = [raw.info['meas_date'] for raw in datasets_EEG]
df = pd.DataFrame({'start': times})

fig, ax = plt.subplots(figsize=(12,6))
ax.scatter(df['start'], [1]*len(df), marker='o')

ax.xaxis.set_major_locator(mdates.DayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
ax.set_yticks([])

plt.title("EEG Recording Sessions by Date")
plt.show()
